In [ ]:
import matplotlib.pyplot as plt
import json
import pandas as pd
from collections import Counter
from lingua import LanguageDetectorBuilder, Language
from tqdm.auto import tqdm
import random
import numpy as np

In [ ]:
f = open(".../20231012_233628_pr_sharings.json")
data = json.loads(f.read())
f.close()

In [ ]:
urls = []
repo_names = []
pull_numbers = []
commits_numbers = []

In [ ]:
comment_urls = []
states = []

In [ ]:
mentioned_property = []
mentioned_text = []

In [ ]:
def generate_documents(data):
    conversation_id = 0
    first_prompts = []
    first_answers = []
    chat_id = 0
    for entry in data["Sources"]:
        for sharedLink in entry["ChatgptSharing"]:
            if sharedLink.get("Conversations"):
                chat_id+=1
                conversation_id = 0
                for conversation in sharedLink["Conversations"]:
                    first_prompts.append({"chat_id" : chat_id, "round_id" : conversation_id, "prompt" : conversation["Prompt"]})
                    first_answers.append({"answer" : conversation["Answer"]})
                    urls.append({"Url" : entry.get("URL")})
                    comment_urls.append({"comment_url" : sharedLink.get("Mention").get("MentionedURL", "N/A")})
                    mentioned_property.append({"MentionedProperty" : sharedLink.get("Mention").get("MentionedProperty", "N/A")})
                    states.append({"states" : entry.get("State", "N/A")})
                    repo_names.append({"RepoName" : entry.get("RepoName")})
                    pull_numbers.append({"PullNumber" : entry.get("Number")})
                    commits_numbers.append({"CommitsNumber" : entry.get("CommitsTotalCount")})
                    mentioned_text.append({"MentionedText" : sharedLink.get("Mention").get("MentionedText", "N/A")})
                    conversation_id+=1
    df = pd.DataFrame(first_prompts).join(pd.DataFrame(first_answers)).reset_index(drop=True)
    df = df.join(pd.DataFrame(states))
    df = df.join(pd.DataFrame(urls))
    df = df.join(pd.DataFrame(comment_urls))
    df = df.join(pd.DataFrame(mentioned_property))
    df = df.join(pd.DataFrame(mentioned_text))
    df = df.join(pd.DataFrame(repo_names))
    df = df.join(pd.DataFrame(pull_numbers))
    df = df.join(pd.DataFrame(commits_numbers))
    return df

In [ ]:
df = generate_documents(data)
df.head(10)

In [ ]:
len(df.groupby("chat_id"))

In [ ]:
len(df)

In [ ]:
#Detect lang https://github.com/pemistahl/lingua-py
def landetec(df):
    detector = LanguageDetectorBuilder.from_all_spoken_languages().build()
    rt_groups = []

    for name, group in df.groupby("chat_id"):
        counter = 0
        not_english_chats_ids = []
        for index, row in group.iterrows():
            lang = detector.detect_language_of(row["prompt"])
            if lang != Language.ENGLISH:
                counter+=1
    #         if row["prompt"].strip() in sample["Prompt"].tolist():
    #             counter += 1
        #some of the rounds are not in english
        if counter == 0:
            rt_groups.append(group)
    #Filter out chats that are not in english
    df_langfiltered = pd.concat(rt_groups)
    return df_langfiltered

In [ ]:
#pr
df_langfiltered = landetec(df)
df_langfiltered.head()

In [ ]:
len(df_langfiltered.groupby("chat_id"))

In [ ]:
len(df_langfiltered)

In [ ]:
df_langfiltered.to_csv("df_langfiltered.csv", encoding = 'utf-8', index = False)

In [ ]:
#df_langfiltered
dropped_duplicates_df_langfiltered = df_langfiltered.drop_duplicates(subset='prompt')
len(dropped_duplicates_df_langfiltered)

In [ ]:
len(dropped_duplicates_df_langfiltered.groupby("chat_id"))

In [ ]:
dropped_duplicates_df_langfiltered.to_csv("dropped_duplicates_pr.csv", encoding = "utf-8", index=False)

In [ ]:
dropped_duplicates_df_langfiltered.to_excel("dropped_duplicates_pr.xlsx", engine = "openpyxl", index=False)